# Im pissed

In [13]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [14]:
import os

df_list = {}

directory = 'Decoded Data Revised'
for filename in os.listdir(directory):
    if filename.endswith('.csv'):
        filepath = os.path.join(directory, filename)
        df_list[filename] = pd.read_csv(filepath)

In [15]:
prefixes = ("Cell", "gnss", "RR", "RL", "FR", "FL")

drop_exact = [
    "Channel", "CAN ID", "Flags", "DLC", "AMSStatus", "APPS1", "APPS2", "Analog8", "BSE", "DisplayKvaserCurrent", "Encoder1",
    "Encoder2", "Encoder3", "Encoder4", "FrontLeft", "FrontLeftPot", "FrontRight",
    "FrontRightPot", "IsolationMonitoring",
    "IsolationMonitoring1", "LeftRad", "PDMBattVoltage", "REGID",
    "RearLeft", "RearLeftPot", "RearRight", "RearRightPot", "RightRad", "PackCurrent", "Data"
]

for df in df_list.values():
    df = df.drop(columns=[col for col in df.columns 
                        if col.startswith(prefixes) or col in drop_exact])

In [16]:
keep_cols = ["Timestamp"]

for df in df_list.values():
    data_cols = [c for c in df.columns if c not in keep_cols]

    df = df[~((df[data_cols].fillna(0) == 0).all(axis=1))]

In [17]:
for key in df_list:
    # Get the DataFrame
    df = df_list[key]
    
    # Convert Timestamp to numeric
    df['Timestamp'] = pd.to_numeric(df['Timestamp'], errors='coerce')
    df = df.set_index('Timestamp')

    # Interpolate all columns based on Timestamp values
    df = df.interpolate(method='values')

    # Reset index to restore Timestamp as a column
    df = df.reset_index()

    # Slice the data and reset index
    df = df.iloc[3750:].reset_index(drop=True)

    # Calculate and apply timestamp offset
    if len(df) > 0:  # Check if DataFrame is not empty
        timestamp_offset = df.loc[0, 'Timestamp']
        df['Timestamp'] = df['Timestamp'] - timestamp_offset
    
    # Save the modified DataFrame back to the dictionary
    df_list[key] = df

In [18]:
# Load the thermistor calibration data
motor_calibration = pd.read_csv(r'Thermistor Plots\motor_thermistors.csv')
controller_calibration = pd.read_csv(r'Thermistor Plots\bamocar_thermistors.csv')

# Convert thermistor values to temperatures for each DataFrame
for df in df_list.values():
    if 'MotorTempRaw' in df.columns:
        df['MotorTemp_C'] = np.interp(df['MotorTempRaw'], 
                                     motor_calibration['Value'], 
                                     motor_calibration['Temperature'])
    if 'ControllerTempRaw' in df.columns:
        df['ControllerTemp_C'] = np.interp(df['ControllerTempRaw'], 
                                          controller_calibration['Value'], 
                                          controller_calibration['Temperature'])

In [19]:
for name, df in df_list.items():
    df = df.set_index('Timestamp')
    # Create a new column for 0.1s bins
    df['timestep'] = (df.index / 0.1).round().astype(int) * 0.1
    # Group by the new timestep and average
    df_10hz = df.groupby('timestep').mean().reset_index().rename(columns={'timestep': 'Timestamp'})
    df_10hz.to_csv('10HZ-Revision/decoded_10Hz_' + name, index=False)